In [1]:
import pandas as pd
import numpy as np
from stravalib import Client
import webbrowser
from Pace import Pace
from units import *
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import webbrowser
from functions import *

In [2]:
client = Client()
client_id = None # Insert your client ID here in int form
client_secret = '[INSERT CLIENT SECRET]'
redirect_uri = 'http://localhost:8282/authorized'

## Generate authorization code

In [3]:
def getCode():
    authorize_url = client.authorization_url(client_id=client_id, 
                                             redirect_uri=redirect_uri,
                                             scope=['read_all', 'profile:read_all', 'activity:read_all'])
    webbrowser.open(authorize_url)
    code = input("Enter the code: ")
    return code

In [4]:
if client.access_token is None:    
    authorize_url = client.authorization_url(client_id=client_id, 
                                             redirect_uri=redirect_uri,
                                             scope=['read_all', 'profile:read_all', 'activity:read_all'])
    
    # Open the authorization URL in a web browser
    webbrowser.open(authorize_url)
    
    # After authorization, you'll be redirected to a URL. Copy the 'code' parameter from this URL
    code = input("Enter the code: ")
    
    # Exchange the code for a token
    token_response = client.exchange_code_for_token(client_id=client_id, 
                                                    client_secret=client_secret, 
                                                    code=code)
    
    # Now you can use the access token to make API requests
    access_token = token_response['access_token']
    client.access_token = access_token

No rates present in response headers


## Get Athlete Profile

In [5]:
activities = client.get_activities()
activity_list = []
for activity in activities:
    activity_dict = {
        'id': activity.id,
        'name': activity.name,
        'start_date': activity.start_date,
        'distance': activity.distance,  # Convert distance to numeric (meters)
        'moving_time': activity.moving_time,  # Convert to seconds
        'elapsed_time': activity.elapsed_time,  # Convert to seconds
        'total_elevation_gain': activity.total_elevation_gain,
        'type': activity.type,
        'average_speed': activity.average_speed if activity.average_speed else None,  # Convert to numeric
        'max_speed': activity.max_speed if activity.max_speed else None,  # Convert to numeric
        'average_heartrate': activity.average_heartrate,
        'max_heartrate': activity.max_heartrate,
    }
    activity_list.append(activity_dict)
myActivities = pd.DataFrame(activity_list)

In [6]:
myActivities

,id,name,start_date,distance,moving_time,elapsed_time,total_elevation_gain,type,average_speed,max_speed,average_heartrate,max_heartrate
0,13399628273,Everything is bigger in Texas,2025-01-19 16:38:43+00:00,13023.6,5079,5079,43.4,root='Run',2.564,4.960,171.9,180.0
1,13382848028,4.5mi Easy Run with Runna ✅,2025-01-17 23:02:43+00:00,5078.0,2178,2206,7.3,root='Run',2.331,5.580,151.7,164.0
2,13374801323,Tempo 2.5mi with Runna ✅,2025-01-16 22:51:13+00:00,6344.0,2358,2580,5.8,root='Run',2.690,4.967,169.3,182.0
3,13366612835,Whole Foods run,2025-01-16 00:04:43+00:00,7137.0,2805,3262,10.0,root='Run',2.544,4.680,156.9,177.0
4,13361372560,Real ones don’t let each other skip leg day,2025-01-15 07:57:16+00:00,0.0,2427,2427,0.0,root='Workout',NaN,NaN,NaN,NaN
5,13357536873,Mile Repeats with Runna ✅,2025-01-14 23:43:45+00:00,8514.1,3408,3436,0.0,root='Run',2.498,4.100,167.8,186.0
6,13342634603,Building Strength with Runna ✅,2025-01-13 07:11:22+00:00,0.0,2379,2379,0.0,root='Workout',NaN,NaN,NaN,NaN
7,13336586068,Winning isn’t easy,2025-01-12 15:46:38+00:00,11428.9,4321,4424,8.4,root='Run',2.645,4.980,171.8,183.0
8,13326411738,Snowy 3 miler,2025-01-11 15:34:14+00:00,5849.1,2030,2071,18.2,root='Run',2.881,4.960,173.3,182.0
9,13315846516,Good morning NYC ☀️,2025-01-10 12:04:44+00:00,6157.0,2436,2725,5.2,root='Run',2.528,5.100,161.4,174.0


## Get Activity Data

In [7]:
def get_activity_streams(client, activity_id, resolution='high'):
    """
    Fetch heart rate, pace (velocity), and elevation streams for a given activity ID.
    """
    streams = client.get_activity_streams(
        activity_id,
        types=['heartrate', 'velocity_smooth', 'altitude', 'time', 'distance'],
        resolution=resolution,
    )
    
    heart_rate = streams['heartrate'].data if 'heartrate' in streams else None
    velocity = streams['velocity_smooth'].data if 'velocity_smooth' in streams else None
    elevation = streams['altitude'].data if 'altitude' in streams else None
    time_index = streams['time'].data if 'time' in streams else None
    distance_index = streams['distance'].data if 'distance' in streams else None
    return heart_rate, velocity, elevation, time_index, distance_index

In [8]:
rowId = 0
things = myActivities.iloc[rowId]
activityId = things['id']
name = things['name']
hr, pace, elevation, timeIdx, distanceIdx = get_activity_streams(client, activityId, resolution='high')

# Convert to imperial system
elevation = [int(round(metersToFeet(elev), 0)) for elev in elevation]
timeIdx = [i/60 for i in timeIdx]
distanceIdx = [round(metersToMiles(i), 2) for i in distanceIdx]

pace = [Pace.from_mps(v) for v in pace if v > 0]

## Interactive Plots

In [10]:
def numericPlot(base, items):
    data = pd.DataFrame({
        base: items,
        'Time': timeIdx,
        'Distance': distanceIdx
    })
    return data

### Pace

In [28]:
len(pace), len(timeIdx)

(2806, 2808)

In [53]:
diff = len(timeIdx) - len(pace)
average = sum(pace)/len(pace)
x = pace + [average] * diff
# x = [-i for i in x]

stuff = numericPlot('Pace', x)
myPaces = stuff['Pace']
index = stuff['Time']

y_values = [pace.time/60 for pace in myPaces]
labels = [str(pace) for pace in myPaces]

# Create the plot
fig = go.Figure(data=go.Scatter(
    x=index,
    y=y_values,
    mode='markers+lines',
    text=labels
))

fig.update_layout(
    title='Pace Plot',
    xaxis_title='Time',
    yaxis_title='Pace (min/mi)'
)
fig.show()

In [54]:
index = stuff['Distance']

y_values = [pace.time/60 for pace in myPaces]
labels = [str(pace) for pace in myPaces]

# Create the plot
fig = go.Figure(data=go.Scatter(
    x=index,
    y=y_values,
    mode='markers+lines',
    text=labels
))

fig.update_layout(
    title='Pace Plot',
    xaxis_title='Distance (mi)',
    yaxis_title='Pace (min/mi)'
)
fig.show()

### HR

In [55]:
zones = client.get_athlete_zones().dict()
values = zones['heart_rate']['zones'][:-1]

In [56]:
def getZone(hr):
    n = len(values)
    assert hr >= 0, 'HR must be non-negative'
    for i in range(n):
        bucket = values[i]
        if bucket['min'] <= hr <= bucket['max']:
            return f'Zone {i+1}'
    raise ValueError(f'HR {hr} is out of range')

In [57]:
base = 'HR'
data = numericPlot(base, hr)
try:
    data['Zone']
except:
    data['Zone'] = data[base].apply(getZone)
fig = go.Figure()

In [58]:
zone_colors = {
    'Zone 1': 'green',
    'Zone 2': 'yellow',
    'Zone 3': 'red',
    'Zone 4': 'red'
}

# Create the figure
fig = go.Figure()

# Add traces for each zone
for zone in data['Zone'].unique():
    zone_data = data[data['Zone'] == zone]
    fig.add_trace(go.Scatter(
        x=zone_data['Time'],
        y=zone_data['HR'],
        mode='lines',
        name=zone,
        line=dict(color=zone_colors[zone], width=1)  # Assign color based on zone
    ))

fig.update_layout(
    title=f'{base}: {name}',
    xaxis_title='Time',
    yaxis_title=base,
    autosize=False,
    width=800,
    height=600
)

fig.show()

In [59]:
# Create the figure
fig = go.Figure()

# Add traces for each zone using Distance as the x-axis
for zone in data['Zone'].unique():
    zone_data = data[data['Zone'] == zone]
    fig.add_trace(go.Scatter(
        x=zone_data['Distance'],
        y=zone_data['HR'],
        mode='lines',
        name=zone,
        line=dict(color=zone_colors[zone], width=1)  # Assign color based on zone
    ))

fig.update_layout(
    title=f'{base}: {name}',
    xaxis_title='Distance',
    yaxis_title=base,
    autosize=False,
    width=800,
    height=600
)

fig.show()

In [44]:
data['Pace'] = x

grouped = data.groupby('Zone').mean()
minimums = data.groupby('Zone').min()
maximums = data.groupby('Zone').max()
for z in grouped.index:
    print(f'{z} Average Pace: {Pace.fromSeconds(grouped.loc[z, "Pace"])}')


Zone 1 Average Pace: 13:00/mi
Zone 2 Average Pace: 9:36/mi
Zone 3 Average Pace: 10:39/mi


In [251]:
# Initialize variables to track the start and end times of each zone
zone_start = None
current_zone = None

# Iterate through the DataFrame to find the start and end times of each zone
for i, row in data.iterrows():
    if row['Zone'] != current_zone:
        if current_zone is not None:
            print(f"{current_zone} from {zone_start} to {data.at[i-1, 'Time']}")
        current_zone = row['Zone']
        zone_start = row['Time']

# Print the last zone
if current_zone is not None:
    print(f"{current_zone} from {zone_start} to {data.at[len(data)-1, 'Time']}")

Zone 1 from 0.0 to 0.45
Zone 2 from 0.48333333333333334 to 1.1833333333333333
Zone 3 from 1.2 to 3.45
Zone 2 from 3.4833333333333334 to 3.4833333333333334
Zone 3 from 3.5166666666666666 to 56.8
Zone 2 from 56.81666666666667 to 56.916666666666664
Zone 3 from 56.95 to 57.18333333333333
Zone 2 from 57.21666666666667 to 57.25
Zone 3 from 57.28333333333333 to 75.43333333333334
Zone 2 from 75.46666666666667 to 75.53333333333333
Zone 3 from 75.56666666666666 to 81.63333333333334
Zone 2 from 81.66666666666667 to 81.7
Zone 3 from 81.73333333333333 to 84.65


### Elevation

In [60]:
data = numericPlot('Elevation', elevation)
fig = px.line(data, x='Time', y='Elevation', title=f'Elevation: {name}')
fig.update_layout(autosize=False, width=800, height=600)
fig.show()